Goal: simulate scenarios according to Costa & Wilkinson-Herbots 2021

In [27]:
import msprime
import numpy as np
from collections import Counter
from dismal.s_matrix import SMatrix
from dismal.inference import DemographicModel

Scenario 1 \
Parameters: [3, 2, 4, 3, 6, 4, 4, 0.2, 0.4, 0.04, 0.08]

In [68]:
# split times
tau0_generations = 8 * 1e6
tau1_generations = 4 * 1e6

# mutation rate per base
mu = 1e-9

# migration rate as decimal of source population
M1_star, M2_star, M1_prime_star, M2_prime_star = [0.2, 0.4, 0.04, 0.08]
m1 = (M1_star/4e6)/2
m2 = (M2_star/2e6)/2
m1_prime = (M1_prime_star/6e6)/2
m2_prime = (M2_prime_star/3e6)/2

In [75]:
demography = msprime.Demography()
demography.add_population(name="a", initial_size=3e6/2)
demography.add_population(name="b1", initial_size=2e6/2)
demography.add_population(name="b2", initial_size=4e6/2)
demography.add_population(name="c1", initial_size=3e6/2)
demography.add_population(name="c2", initial_size=6e6/2)
demography.add_population_split(time=tau0_generations, derived=["b1", "b2"], ancestral="a")
demography.add_population_split(time=tau1_generations, derived=["c1"], ancestral="b1")
demography.add_population_split(time=tau1_generations, derived=["c2"], ancestral="b2")
demography.set_migration_rate("b2", "b1", m1) # NB: backwards in time in msprime
demography.set_migration_rate("b1", "b2", m2)
demography.set_migration_rate("c2", "c1", m1_prime)
demography.set_migration_rate("c1", "c2", m2_prime)
demography.sort_events()

In [76]:
block_len = 500
num_replicates = 10000

ts_state1 = msprime.sim_ancestry(samples={'c1':2, 'c2':0}, demography=demography, sequence_length=block_len, num_replicates=num_replicates, ploidy=2)
ts_state2 = msprime.sim_ancestry(samples={'c1':0, 'c2':2}, demography=demography, sequence_length=block_len, num_replicates=num_replicates, ploidy=2)
ts_state3 = msprime.sim_ancestry(samples={'c1':1, 'c2':1}, demography=demography, sequence_length=block_len, num_replicates=num_replicates, ploidy=2)

In [77]:
s = []
for ts in [ts_state1, ts_state2, ts_state3]:
    sim = []
    for replicate_index, ts in enumerate(ts):
        ts_muts = msprime.sim_mutations(ts, rate=mu, discrete_genome=False)
        sim.append(int(ts_muts.divergence(sample_sets=[[0],[2]], span_normalise=False)))
    s.append(sim)


In [78]:
S = SMatrix().from_lists(s)

In [79]:
gim = DemographicModel(
    pop_size_change=True,
    mig_rate_change=True,
    migration=True,
    asymmetric_m_prime_star=True,
    asymmetric_m_star=True
)
gim


        DISMaL DemographicModel object.
        Theta (population size) parameters: ['theta0', 'theta1', 'theta2', 'theta1_prime', 'theta2_prime']
        Tau (split time) parameters: ['t1', 'v']
        M (migration rate) parameters: ['M1_star', 'M2_star', 'M1_prime_star', 'M2_prime_star']
        

In [80]:
gim.infer(S, n_iterations=1)

100%|██████████| 1/1 [00:23<00:00, 23.02s/it]


({'theta0': 2.909710980121525,
  'theta1': 2.8521126317622394,
  'theta2': 3.1582422688451683,
  'theta1_prime': 3.0743148302007985,
  'theta2_prime': 5.742952530525443,
  't1': 4.881966514393909,
  'v': 3.2642714178022865,
  'M1_star': 0.916302824286449,
  'M2_star': 0.3635309693298617,
  'M1_prime_star': 0.04212627889786452,
  'M2_prime_star': 0.08135152367709311},
 78974.45056100015,
 11)

In [51]:
! ls ~/Dropbox/DISMaL_chapter/simulations/msprime_initial_tests

In [64]:
def s_list_to_csv(s_list, path):
    with open(path, "w") as f:
        [f.write(str(s[0][i]) + ",") for i in range(len(s[0]))]

In [66]:
[s_list_to_csv(s[i-1], f"../../simulations/msprime_initial_tests/scenario1_x{i}.csv") for i in range(1,4)]

[None, None, None]

In [67]:
S

array([[1.396e+03, 1.142e+03, 9.810e+02, 8.110e+02, 6.830e+02, 6.220e+02,
        5.110e+02, 4.230e+02, 3.460e+02, 2.890e+02, 2.410e+02, 2.130e+02,
        1.840e+02, 1.420e+02, 1.300e+02, 1.190e+02, 7.700e+01, 8.700e+01,
        7.400e+01, 5.500e+01, 6.200e+01, 5.100e+01, 3.700e+01, 4.100e+01,
        3.400e+01, 2.600e+01, 3.400e+01, 3.800e+01, 2.800e+01, 3.100e+01,
        2.500e+01, 2.500e+01, 3.300e+01, 1.800e+01, 2.700e+01, 2.800e+01,
        2.000e+01, 2.600e+01, 2.900e+01, 4.800e+01, 2.800e+01, 3.300e+01,
        3.000e+01, 4.000e+01, 3.400e+01, 4.400e+01, 3.800e+01, 2.900e+01,
        4.500e+01, 4.100e+01, 4.000e+01, 3.900e+01, 2.800e+01, 3.000e+01,
        4.400e+01, 3.100e+01, 4.000e+01, 1.800e+01, 3.000e+01, 2.300e+01,
        1.400e+01, 1.700e+01, 6.000e+00, 1.400e+01, 8.000e+00, 1.100e+01,
        1.300e+01, 5.000e+00, 6.000e+00, 4.000e+00, 3.000e+00, 4.000e+00,
        5.000e+00, 4.000e+00, 2.000e+00, 1.000e+00, 1.000e+00, 3.000e+00,
        0.000e+00, 1.000e+00, 1.000e+0